In [2]:
adj_matrix = list([
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 1],
    [1, 1, 0, 0, 1, 1, 0, 1],
    [0, 1, 0, 0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 1, 0, 0, 0],
])

In [8]:
# Шлях до вашого текстового файлу
file_path = f"C:\\Users\\julia\\projects\\Intelligent_analysis_of_complex_networks\\var01 (1).txt"

# Зчитуємо файл
with open(file_path, "r") as f:
    data = f.read().strip()

# Перетворюємо рядок на список чисел
binary_list = [int(char) for char in data]

# Визначаємо розмір матриці
N = int(len(binary_list) ** 0.5)

# Формуємо матрицю суміжності
matrix_256x256 = []
for i in range(N):
    row = binary_list[i * N:(i + 1) * N]  # Формуємо рядок матриці
    matrix_256x256.append(row)

print(f"Останній символ в матриці 256x256: {matrix_256x256[255][255]}")

Останній символ в матриці 256x256: 0


In [11]:
def analyze_network(adj_matrix):
    N = len(adj_matrix)  # кількість вузлів

    # 1. Матриця відстаней (алгоритм Флойда-Варшалла вручну)
    inf = float('inf')
    distance_matrix = [[inf] * N for _ in range(N)]
    for i in range(N):
        for j in range(N):
            if i == j:
                distance_matrix[i][j] = 0
            elif adj_matrix[i][j] == 1:
                distance_matrix[i][j] = 1

    for k in range(N):
        for i in range(N):
            for j in range(N):
                if distance_matrix[i][k] < inf and distance_matrix[k][j] < inf:
                    distance_matrix[i][j] = min(distance_matrix[i][j], distance_matrix[i][k] + distance_matrix[k][j])

    # Перетворимо нескінченність на None для кращого відображення
    print("Матриця відстаней:")
    for i in range(N):
        print([int(d) if d != inf else None for d in distance_matrix[i]])

    # 2. Діаметр мережі (максимальна відстань між усіма вузлами)
    diameter = max(max(row[j] for j in range(N) if row[j] != inf) for row in distance_matrix)

    # 3. Середня довжина найкоротших шляхів
    sum_distances = sum(distance_matrix[i][j] for i in range(N) for j in range(N) if i != j and distance_matrix[i][j] != inf)
    num_pairs = sum(1 for i in range(N) for j in range(N) if i != j and distance_matrix[i][j] != inf)
    average_shortest_path_length = sum_distances / num_pairs

    # 4. Транзитивність (підрахунок кількості трикутників і трійок)
    triangles = 0  # Кількість трикутників
    triplets = 0  # Кількість трійок (відкритих та замкнутих)

    for i in range(N):
        neighbors_i = [j for j in range(N) if adj_matrix[i][j] == 1]
        for j in range(len(neighbors_i)):
            for k in range(j + 1, len(neighbors_i)):
                node_j = neighbors_i[j]
                node_k = neighbors_i[k]
                if adj_matrix[node_j][node_k] == 1:
                    triangles += 1  # Замкнутий трикутник
                triplets += 1  # Відкрита або замкнута трійка

    transitivity = (triangles / triplets) if triplets > 0 else 0

    # 5. Центральність вузлів (наприклад, центральність посередництва)
    node_centralities = []
    for i in range(N):
        centrality = sum(1 / distance_matrix[i][j] for j in range(N) if i != j and distance_matrix[i][j] < inf)
        node_centralities.append(centrality)

    # 6. Топ-10 вузлів за центральністю
    top_10_nodes = sorted(
        [(i + 1, node_centralities[i]) for i in range(N)],
        key=lambda x: x[1], reverse=True
    )[:10]

    # 7. Показник центральності мережі
    max_centrality = max(node_centralities)
    sum_differences = sum(max_centrality - c for c in node_centralities)
    network_centrality = sum_differences / ((N - 1) * (N - 2) / 2) if N > 2 else 0

    # 8. Власний вектор та власне значення (центральність власного вектора)
    eigen_centrality = [1] * N  # Початковий вектор центральності
    epsilon = 1e-4  # Точність
    max_iterations = 1000
    lambda_value = 0

    for _ in range(max_iterations):
        next_centrality = [0] * N
        for i in range(N):
            for j in range(N):
                next_centrality[i] += adj_matrix[i][j] * eigen_centrality[j]
        # Нормалізація
        norm = max(next_centrality)
        if norm == 0:  # Якщо немає зв’язків
            lambda_value = 0
            break
        next_centrality = [x / norm for x in next_centrality]

        # Перевірка на збіжність
        if all(abs(next_centrality[i] - eigen_centrality[i]) < epsilon for i in range(N)):
            lambda_value = norm
            break
        eigen_centrality = next_centrality

    # Повернення результатів
    return {
        "Матриця відстаней": distance_matrix,
        "Діаметр мережі": diameter,
        "Середня довжина найкоротших шляхів": average_shortest_path_length,
        "Транзитивність мережі": transitivity,
        "Центральність вузлів": node_centralities,
        "Топ-10 вузлів за центральністю": top_10_nodes,
        "Показник центральності мережі": network_centrality,
        "Центральність власного вектора": eigen_centrality,
        "Власне значення (λ)": lambda_value
    }



In [13]:
results = analyze_network(adj_matrix)
print("\nРезультати аналізу мережі:")
for key, value in results.items():
    print(f"{key}: {value}")

Матриця відстаней:
[0, 2, 1, 3, 2, 1, 4, 2]
[2, 0, 1, 1, 1, 2, 2, 1]
[1, 1, 0, 2, 1, 1, 3, 1]
[3, 1, 2, 0, 1, 3, 1, 2]
[2, 1, 1, 1, 0, 2, 2, 1]
[1, 2, 1, 3, 2, 0, 4, 2]
[4, 2, 3, 1, 2, 4, 0, 3]
[2, 1, 1, 2, 1, 2, 3, 0]

Результати аналізу мережі:
Матриця відстаней: [[0, 2, 1, 3, 2, 1, 4, 2], [2, 0, 1, 1, 1, 2, 2, 1], [1, 1, 0, 2, 1, 1, 3, 1], [3, 1, 2, 0, 1, 3, 1, 2], [2, 1, 1, 1, 0, 2, 2, 1], [1, 2, 1, 3, 2, 0, 4, 2], [4, 2, 3, 1, 2, 4, 0, 3], [2, 1, 1, 2, 1, 2, 3, 0]]
Діаметр мережі: 4
Середня довжина найкоротших шляхів: 1.8571428571428572
Транзитивність мережі: 0.6
Центральність вузлів: [4.083333333333333, 5.5, 5.833333333333333, 4.666666666666667, 5.5, 4.083333333333333, 3.1666666666666665, 4.833333333333333]
Топ-10 вузлів за центральністю: [(3, 5.833333333333333), (2, 5.5), (5, 5.5), (8, 4.833333333333333), (4, 4.666666666666667), (1, 4.083333333333333), (6, 4.083333333333333), (7, 3.1666666666666665)]
Показник центральності мережі: 0.4285714285714285
Центральність власного вектор

In [14]:
results = analyze_network(matrix_256x256)
print("\nРезультати аналізу мережі:")
for key, value in results.items():
    print(f"{key}: {value}")

Матриця відстаней:
[0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 3, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 3, 1, 2, 2, 1, 1, 1, 2, 3, 2, 2, 2, 1, 2, 3, 2, 1, 1, 2, 2, 3, 1, 2, 1, 2, 2, 2, 1, 2, 3, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 3, 2, 2, 2, 2, 2, 1, 3, 1, 3, 2, 3, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 3, 1, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3, 2, 4, 1, 2, 2, 4, 5, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 1, 2, 1, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, 1, 3, 2, 2, 1, 3, 3, 2, 3, 2, 2, 2, 2, 1, 2, 1, 3, 1, 3, 2, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 3, 3, 2, 1, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]
[1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 3, 1, 2, 2, 1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 3, 1, 1, 2, 1, 1

In [33]:
def calculate_node_centrality(adj_matrix):
    N = len(adj_matrix)  # Кількість вузлів

    # Ініціалізація центральності кожного вузла
    centrality = [0] * N  # Центральність за посередництвом (міра посередництва)

    # Ініціалізуємо матрицю найкоротших шляхів і кількість шляхів
    inf = float('inf')
    distance_matrix = [[inf] * N for _ in range(N)]
    path_count = [[0] * N for _ in range(N)]

    # Заповнюємо матрицю найкоротших шляхів
    for i in range(N):
        for j in range(N):
            if i == j:
                distance_matrix[i][j] = 0
                path_count[i][j] = 1
            elif adj_matrix[i][j] == 1:
                distance_matrix[i][j] = 1
                path_count[i][j] = 1

    # Алгоритм Флойда-Варшалла для пошуку найкоротших шляхів
    for k in range(N):
        for i in range(N):
            for j in range(N):
                if distance_matrix[i][j] > distance_matrix[i][k] + distance_matrix[k][j]:
                    distance_matrix[i][j] = distance_matrix[i][k] + distance_matrix[k][j]
                    path_count[i][j] = path_count[i][k] * path_count[k][j]
                elif distance_matrix[i][j] == distance_matrix[i][k] + distance_matrix[k][j]:
                    path_count[i][j] += path_count[i][k] * path_count[k][j]

    # Обчислення центральності за посередництвом
    for node in range(N):
        for start in range(N):
            for end in range(N):
                if start != end and start != node and end != node:
                    # Перевіряємо, чи вузол node лежить на найкоротшому шляху
                    if distance_matrix[start][node] + distance_matrix[node][end] == distance_matrix[start][end]:
                        centrality[node] += (
                            path_count[start][node] * path_count[node][end] / path_count[start][end]
                        )

    # Нормалізація центральності
    max_centrality = (N - 1) * (N - 2) / 2
    normalized_centrality = [c / max_centrality for c in centrality]

    return {
        "Центральність вузлів": centrality,
        "Нормована центральність": normalized_centrality
    }


In [34]:
centrality_results = calculate_node_centrality(adj_matrix)
print("\nРезультати обчислення центральності вузлів:")
for key, value in centrality_results.items():
    print(f"{key}: {value}")


Результати обчислення центральності вузлів:
Центральність вузлів: [0, 4.681069958847736, 42.23569023569022, 15.96296296296296, 11.8008658008658, 0, 0, 0]
Нормована центральність: [0.0, 0.22290809327846364, 2.011223344556677, 0.7601410934744267, 0.5619459905174191, 0.0, 0.0, 0.0]


In [35]:
centrality_results = calculate_node_centrality(matrix_256x256)
print("\nРезультати обчислення центральності вузлів:")
for key, value in centrality_results.items():
    print(f"{key}: {value}")


Результати обчислення центральності вузлів:
Центральність вузлів: [2995.306523541571, 10363.098790138527, 96.55011249123984, 1834.0270886560756, 11552.637113886554, 79.85029747816263, 5045.025221971687, 1083.4995232267142, 1331.2430336386083, 0, 4857.040971035084, 94.72570140202288, 0, 0, 1684.2670711035698, 981.6493038397953, 2741.9036726927175, 0, 156.45942046584966, 1510.5566286446635, 37.52611039259317, 375.8709168193462, 205.7047643943802, 250.86027784833095, 0, 0, 38.115237668578565, 0, 0, 38.115237668578565, 0, 1461.2677977447922, 81.5606210256877, 0, 321.9694870356678, 195.41797753503548, 324.65945083769, 39.141881106812406, 0, 0, 0, 449.0503258850005, 0, 304.6648266161772, 39.22613286536607, 0, 39.22613286536607, 0, 0, 0, 0, 204.32742784561398, 156.69647796119102, 0, 0, 0, 0, 0, 39.331098595053355, 467.8070496852462, 851.7593625347088, 153.8980341006124, 1057.2334763604147, 0, 40.34526786152662, 68.2718636431577, 40.34526786152662, 0, 1321.4079633022736, 727.7839237368406, 40

In [36]:
def calculate_network_centrality(adj_matrix):
    N = len(adj_matrix)  # Кількість вузлів

    # Розраховуємо центральність вузлів за мірою посередництва
    node_centralities = calculate_node_centrality(adj_matrix)
    centralities = node_centralities["Центральність вузлів"]

    # Сума центральностей вузлів
    sum_centralities = sum(centralities)

    # Максимально можлива центральність у мережі-"зірка"
    # У мережі-"зірка" один вузол є посередником для всіх інших
    star_max_centrality = (N - 1) * (N - 2) / 2  # Центральність центрального вузла у зірці
    star_total_centrality = star_max_centrality * (N - 1)  # Сумарна центральність у зірці

    # Розраховуємо нормовану центральність мережі
    if star_total_centrality > 0:
        network_centrality = sum_centralities / star_total_centrality
    else:
        network_centrality = 0

    return {
        "Сумарна центральність мережі": sum_centralities,
        "Нормована центральність мережі": network_centrality,
        "Максимальна центральність вузла у зірці": star_max_centrality,
        "Сумарна центральність у зірці": star_total_centrality
    }


In [40]:
results = calculate_network_centrality(adj_matrix)
for key, value in results.items():
    print(f"{key}: {value}")

Сумарна центральність мережі: 74.68058895836671
Нормована центральність мережі: 0.5080312174038552
Максимальна центральність вузла у зірці: 21.0
Сумарна центральність у зірці: 147.0


In [41]:
results = calculate_network_centrality(matrix_256x256)
for key, value in results.items():
    print(f"{key}: {value}")

Сумарна центральність мережі: 76844.50519770561
Нормована центральність мережі: 0.00930526480701918
Максимальна центральність вузла у зірці: 32385.0
Сумарна центральність у зірці: 8258175.0
